In [163]:
from functools import cmp_to_key
import os
import pandas as pd

In [164]:
def compare_file(file1, file2):
    num1 = int(file1.split('.')[0])  
    num2 = int(file2.split('.')[0])  
    if num1 < num2:
        return -1
    elif num1 > num2:
        return 1
    else:
        return 0
def pb_read_appendix1(path="./Problem B/Appendix 1"):
   data=[]
   for (_, _, filenames) in os.walk(path):
      temp_save=filenames.copy()
      for idx, ff in enumerate(filenames):
          if ff.startswith("~"):
              temp_save.remove(ff)
      filenames=sorted(temp_save, key=cmp_to_key(compare_file))
      for filename in filenames:
         excel_instance=pd.read_excel(os.path.join(path, filename), sheet_name='Sheet1')
         data.append({})
         data[-1]["date"]=[i for i in excel_instance["Var1"].to_list()]
         data[-1]["time"]=[i for i in excel_instance["Var2"].to_list()]
         data[-1]["x"]=[i for i in excel_instance["Var3"].to_list()]
         data[-1]["y"]=[i for i in excel_instance["Var4"].to_list()]
         data[-1]["z"]=[i for i in excel_instance["Var5"].to_list()]
         print(f"{filename} len:{len(data[-1]["date"])}")
   lower_range=555 
   upper_range=1033
   for sensor in data:
       sensor["date"]=sensor["date"][lower_range:upper_range+1]
       sensor["time"]=sensor["time"][lower_range:upper_range+1]
       sensor["x"]=sensor["x"][lower_range:upper_range+1]
       sensor["y"]=sensor["y"][lower_range:upper_range+1]
       sensor["z"]=sensor["z"][lower_range:upper_range+1]
   return data

In [165]:
appendix1=pb_read_appendix1() #读取所有传感器的数据到一个字典里 只取24:00到4:00的数据！

1.xlsx len:4310
2.xlsx len:4310
3.xlsx len:4310
4.xlsx len:4310
5.xlsx len:4310
6.xlsx len:4310
7.xlsx len:4310
8.xlsx len:4310
9.xlsx len:4310
10.xlsx len:4310
11.xlsx len:4310
12.xlsx len:4310
13.xlsx len:4310
14.xlsx len:4310
15.xlsx len:4310
16.xlsx len:4310
17.xlsx len:4310
18.xlsx len:4310
19.xlsx len:4310
20.xlsx len:4310
21.xlsx len:4310
22.xlsx len:4310
23.xlsx len:4310
24.xlsx len:4310
25.xlsx len:4310
26.xlsx len:4310
27.xlsx len:4310
28.xlsx len:4310
29.xlsx len:4310
30.xlsx len:4310
31.xlsx len:4310
32.xlsx len:4310
33.xlsx len:4310
34.xlsx len:4310
35.xlsx len:4310
36.xlsx len:4310
37.xlsx len:4310
38.xlsx len:4310
39.xlsx len:4310
40.xlsx len:4310
41.xlsx len:4310
42.xlsx len:4310
43.xlsx len:4310
44.xlsx len:4310
45.xlsx len:4310
46.xlsx len:4310
47.xlsx len:4310
48.xlsx len:4310
49.xlsx len:4310
50.xlsx len:4310
51.xlsx len:4310
52.xlsx len:4310
53.xlsx len:4310
54.xlsx len:4310
55.xlsx len:4310
56.xlsx len:4310
57.xlsx len:4310
58.xlsx len:4310
59.xlsx len:4310
60.xls

# 读取格式
读取出来是一个array,每个array是每个传感器的信息
## keys 
1. date$\Rightarrow$一个包含不同点日期的array
2. time$\Rightarrow$一个包含对应时间戳的点
3. x$\Rightarrow$该时间戳下的x值
4. y$\Rightarrow$该时间戳下的y值
5. z$\Rightarrow$该时间戳下的z值

In [ ]:
import torch 
import numpy as np
device="cuda"

In [ ]:
xs=[]
ys=[]
zs=[]
for sensor in appendix1:
    xs.append([x-sensor["x"][0] for x in sensor["x"]])
    ys.append([y-sensor["y"][0] for y in sensor["y"]])
    zs.append([z-sensor["z"][0] for z in sensor["z"]])
xs=np.array(xs)
ys=np.array(ys)
zs=np.array(zs)
xs=torch.from_numpy(xs).to(device)
ys=torch.from_numpy(ys).to(device)
zs=torch.from_numpy(zs).to(device)


torch.Size([353, 479])


In [244]:
num_sensors=353
num_time_stamps=479
xyzs=torch.stack([xs,ys,zs])
xyzs=xyzs.permute(2,1,0)
xyz_a=xyzs.unsqueeze(2).repeat(1,1,num_sensors,1)
xyz_b=xyz_a.transpose(1,2)
diff=xyz_a-xyz_b
diff=torch.abs(diff)
print(diff.size())
diff=torch.sum(diff,3)
print(diff.size())
diff=torch.sum(diff,2)
print(diff.size())
res_idx=torch.argmin(diff,1)
print(res_idx.size())
best_sensors=[]
for idx, data in enumerate(res_idx):
    print(f"Time:{appendix1[0]["time"][idx]}, Best Sensor is No.{data.item()+1}")
    best_sensors.append(data.item()+1)

torch.Size([479, 353, 353, 3])
torch.Size([479, 353, 353])
torch.Size([479, 353])
torch.Size([479])
Time:00:00:36, Best Sensor is No.1
Time:00:01:06, Best Sensor is No.327
Time:00:01:36, Best Sensor is No.252
Time:00:02:06, Best Sensor is No.185
Time:00:02:36, Best Sensor is No.40
Time:00:03:06, Best Sensor is No.81
Time:00:03:36, Best Sensor is No.81
Time:00:04:06, Best Sensor is No.238
Time:00:04:36, Best Sensor is No.347
Time:00:05:06, Best Sensor is No.115
Time:00:05:36, Best Sensor is No.347
Time:00:06:06, Best Sensor is No.180
Time:00:06:36, Best Sensor is No.55
Time:00:07:06, Best Sensor is No.2
Time:00:07:36, Best Sensor is No.115
Time:00:08:06, Best Sensor is No.89
Time:00:08:36, Best Sensor is No.303
Time:00:09:06, Best Sensor is No.11
Time:00:09:36, Best Sensor is No.294
Time:00:10:06, Best Sensor is No.147
Time:00:10:36, Best Sensor is No.61
Time:00:11:06, Best Sensor is No.233
Time:00:11:36, Best Sensor is No.300
Time:00:12:06, Best Sensor is No.50
Time:00:12:36, Best Sens

In [246]:
result1=pd.DataFrame(data={"Time":appendix1[0]["time"], "Best sensor":best_sensors})

In [249]:
with pd.ExcelWriter("Problem B_1.xlsx", engine="openpyxl") as writer:
    result1.to_excel(writer)